In [ ]:
import pandas as pd
import gzip
import os
from geopy.geocoders import Nominatim
from geopy.point import Point
import geopandas as gpd
from shapely.geometry import Point

---

<h1><center>Bussiness Google</center></h1>

<h1><center>Lectura de información</center></h1>

El primer paso antes de realizar la normalización y limpieza de los datasets correspondientes a los metadatos de google, fue unificar los ya mencionados datasets para luego proceder a realizar la exportación de los mismos.

In [ ]:
# En este bloque, realize la apertura de todos los .json de meta datos de google, para luego concatenarlos y formar un solo dataframe con toda la informacion
'''
dicc = {}

for x in range(1,12):
    dicc[x] = pd.read_json(f'extras\Google Maps\metadata-sitios\{x}.json', lines=True)
    
# Lista para almacenar los DataFrames
lista_dataframes = []

# Iterar sobre los valores del diccionario (los Datasets<.json>)
for df in dicc.values():
    lista_dataframes.append(df)

# Concatenar los DataFrames en uno solo
resultado_concatenado = pd.concat(lista_dataframes, ignore_index=True)

'''

### Cabe aclarar que este bloque de comandos solo se utiliza en primera instancia, ya que luego se trabajara sobre la exportación unificada

In [ ]:
#resultado_concatenado.head()

In [ ]:
# Lectura de dataset unificado  
meta_datos_google = pd.read_json(r'..\extras\google Maps\metadata-sitios\bussiness_google.json.gz')

meta_datos_google.head()

In [ ]:
print(resultado_concatenado.columns.values)

## Columnas eliminadas

-state (porque no es relevante, dice si esta abierto el local o no)

-url (se puede ingresar utilizando la siguiente url mas el gmap-id) [https://www.google.com/maps/place//data=!4m2!3m1!1s {aca va el gmap_id}]

-description (esta información podemos obtenerla en caso de ser necesario usando la api, solo tiene 250.000 datos lo que vuelve a la columna irrelevante en comparación con la cantidad total, por eso mismo se recomienda realizar la extracción de dicha columna de ser necesario desde la api)

-MISC (esta columna tiene una descripcion de como estan las condiciones del local)

-price (es irrelevante el precio)

-num_of_reviews (esta columna tendrémos que volver a crearla luego de realizar el ETL de los estados)

-address (esta información podemos extraerla del url en el caso de ser necesario)

-hours (en caso de ser necesario se extraeran los datos específicos de la url)

-relative_result (podemos obtener el mismo resultado y hasta mas eficiente realizando una ponderación de los valores de latitude y longitude)

Muestra de acceso a google maps por medio de url mas gmap_id:

https://www.google.com/maps/place//data=!4m2!3m1!1s0x88e77e406e2da09d:0x479c876de5fd4ec4

In [ ]:
resultado_concatenado.drop(columns=['state','url','description','MISC','price','num_of_reviews','address','hours','relative_results'],inplace=True)

In [ ]:
resultado_concatenado

In [ ]:
bussiness_google = pd.read_json(r'..\datasets\processed\google\bussiness_google.json.gz')
bussiness_google

In [ ]:
illinois = gpd.read_file(r'..\datasets\raw\IL.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_illinois_google = gpd.sjoin(meta_datos_google, illinois, op='within')

In [ ]:
california = gpd.read_file(r'..\datasets\raw\CA.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_california_google = gpd.sjoin(meta_datos_google, california, op='within')

In [ ]:
florida = gpd.read_file(r'..\datasets\raw\FL.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_florida_google = gpd.sjoin(meta_datos_google, florida, op='within')

In [ ]:
Nuevajersey = gpd.read_file(r'..\datasets\raw\NJ.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_Nuevajersey_google = gpd.sjoin(meta_datos_google, Nuevajersey, op='within')

In [ ]:
print(len(empresas_en_florida_google) + len(empresas_en_california_google) + len(empresas_en_illinois_google) + len(empresas_en_Nuevajersey_google))

In [ ]:
# Lista de DataFrames a concatenar
dataframes = [empresas_en_florida_google, empresas_en_california_google, empresas_en_illinois_google, empresas_en_Nuevajersey_google]

# Concatenar los DataFrames verticalmente
empresas_concatenadas_google = pd.concat(dataframes, ignore_index=True)

In [ ]:
# Lista de nombres de columnas a eliminar
columnas_a_eliminar = ['geometry', 'index_right', 'id', 'fips','Estado_Perteneciente']

# Eliminar las columnas de empresas_concatenadas
empresas_concatenadas_google = empresas_concatenadas_google.drop(columns=columnas_a_eliminar)

empresas_concatenadas_google.rename(columns={'name_right': 'state'}, inplace=True)
empresas_concatenadas_google.rename(columns={'name_left': 'name'}, inplace=True)
empresas_concatenadas_google.rename(columns={'avg_rating': 'stars'}, inplace=True)

empresas_concatenadas_google.head()

In [ ]:
# Creo listas para utilizar en las siguientes iteraciones
categorias = []
filtro_cat = []
restaurant_cat = []

# Itero por la columna categories del dataframe, para extraer todos sus valores y convertirlos en listas, los que luego agrego a la lista categorias
for x in empresas_concatenadas_google['category']:
    alm = x.split("'")
    categorias.append(alm)
    
# Realizo una limpieza de cada lista dentro de la lista categorias
for x in categorias:
    for i in x:
        if len(i) > 3:
            filtro_cat.append(i)

# Itero sobre cada fila de la columna categories, con la finalidad de filtrar todos los comercios que corresponden a restaurantes
for x in filtro_cat:
    if 'estau' in x:
        restaurant_cat.append(x)

# Con esta conversion elimino los valores repetidos de toda mi lista
restaurant_cat = set(restaurant_cat)

# Creo un filtro con el cual selecciono las columnas coincidentes
filtro = empresas_concatenadas_google['category'].str.contains('|'.join(restaurant_cat))

# Exporto el dataframe con su nueva mascara para crear uno nuevo con los datos esperados
empresas_concatenadas_restaurantes_estados_google = empresas_concatenadas_google[filtro]

In [ ]:
empresas_concatenadas_restaurantes_estados_google

Normalizamos los valores de categorias en formato lista.

In [ ]:
import ast
empresas_concatenadas_restaurantes_estados_google['category'] = empresas_concatenadas_restaurantes_estados_google['category'].apply(lambda x: x.replace("\n", "").replace("' '", "', '").replace('" ', '", ').replace(' "', ', "').replace(',,', ','))
empresas_concatenadas_restaurantes_estados_google['category'] = empresas_concatenadas_restaurantes_estados_google['category'].apply(lambda x: ast.literal_eval(x))

In [ ]:
#Normalizamos los nombres de las columnas y actualizo index
empresas_concatenadas_restaurantes_estados_google = empresas_concatenadas_restaurantes_estados_google[['gmap_id', 'name', 'latitude', 'longitude', 'stars', 'category', 'state']]
empresas_concatenadas_restaurantes_estados_google.columns = ['gmap_id', 'name', 'latitude', 'longitude', 'stars', 'categories', 'state']
empresas_concatenadas_restaurantes_estados_google.reset_index(drop=True, inplace=True)

empresas_concatenadas_restaurantes_estados_google.to_parquet(r'..\datasets\processed\google\bussiness_google.parquet.gz', compression='gzip')